In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import math
import scipy
from scipy import stats

In [3]:
os.getcwd()

'C:\\Users\\admin\\Desktop\\Python Projects\\AB Test'

In [4]:
os.chdir('C:\\Users\\admin\\Desktop\\Python Projects\\AB Test')

In [5]:
os.getcwd()

'C:\\Users\\admin\\Desktop\\Python Projects\\AB Test'

# Experiment Overview: Free Trial Screener
## (Udacity Project)

At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials". If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first. If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.


In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course. If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free. At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. 

The hypothesis was that this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time—without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.


The unit of diversion is a cookie, although if the student enrolls in the free trial, they are tracked by user-id from that point forward. The same user-id cannot enroll in the free trial twice. For users that do not enroll, their user-id is not tracked in the experiment, even if they were signed in when they visited the course overview page.

# Step 1: Metric Choice

## Invariant Metrics

The purpose of invariant metrics is to ensure that the experiment setup is correct. The metrics chosen are not supposed to be affected by the experiment and should not differ between the control and experiment groups

* **Number of cookies:** Number of unique cookies to view the course overview page <br>
* **Number of clicks:** Number of unique cookies to click the "Start free trial" button <br>
* **Click-through-probability:** Number of unique cookies to click the "Start free trial" button divided by the number of unique cookies to view the course overview page

## Evaluation Metrics

Metrics that are expected to change after the experiment, which are used to determine if our hypothesis is valid. The hypothesis is that if students who clicked the "start free trial" were asked how much time they had to devote to the course and had more clear expectation set, then more students would continue past the free trial and eventually complete the course

* **Gross Conversion:** Number of user-ids to complete checkout and enroll in the free trial divided by the number of unique cookies to click the "Start free trial" button <br>
* **Retention:** Number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to co mplete checkout <br>
* **Net Conversion:** Number of user-ids to remain enrolled past the 14-day boundary (and thurs make at least one payment) divided by the number of unique cookies to click the "Start free trial" button

# Step 2: Calculating Standard Deviation

### Reading in Data

In [24]:
xls = pd.ExcelFile('C:\\Users\\admin\\Desktop\\Python Projects\\AB Test\\Final Project Results.xlsx')
xls2 = pd.ExcelFile('C:\\Users\\admin\\Desktop\\Python Projects\\AB Test\\Final Project Baseline Values.xlsx')
baseline = pd.read_excel(xls2,'Sheet1',header = None, names = ['metric','value'])
control = pd.read_excel(xls,'Control')
experiment = pd.read_excel(xls,'Experiment')

In [25]:
baseline.head(10)

,metric,value
0,Unique cookies to view course overview page pe...,40000.000000
1,"Unique cookies to click ""Start free trial"" per...",3200.000000
2,Enrollments per day:,660.000000
3,"Click-through-probability on ""Start free trial"":",0.080000
4,"Probability of enrolling, given click:",0.206250
5,"Probability of payment, given enroll:",0.530000
6,"Probability of payment, given click",0.109313


In [26]:
control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [9]:
experiment.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


### Distribution

Distribution is assumed to be Binomial, which will be a normal distribution for the sample sizes. <br>

Would typically look at the distribution of the underlying data used for the baseline in order to determine the distribution

### Calculation

Baseline values are for 40,000 pageviews, but our sample groups are composed of only 5,000 cookies. In order to scale the baseline values to match the groups they can be scaled down by a factor (Number of pageviews in samples/Number of all pageviews in a day)

Standard Deviation = sqrt(p(1-p)/N) <br>
p = probability of metrics <br>
n = number of users <br>

### Standard Deviation for Evaluation Metrics

In [10]:
# Gross Conversion
pgc = .20625
n = .08
pgc_std = math.sqrt(pgc*(1-pgc)/(5000*n))
print('Gross Conversion Standard Deviation = ', pgc_std)

Gross Conversion Standard Deviation =  0.020230604137049392


In [11]:
# Retention
pr = .53
n = .08 * .20625
pr_std = math.sqrt(pr*(1-pr)/(5000*n))
print('Retention Standard Deviation = ', pr_std)

Retention Standard Deviation =  0.05494901217850908


In [12]:
# Net Conversion
pnc = .109313
n = .08
pnc_std = math.sqrt(pnc*(1-pnc)/(5000*n))
print('Net Conversion Standard Deviation = ', pnc_std)

Net Conversion Standard Deviation =  0.015601575884425905


# Step 3: Determining the number of Pageviews Needed for Experiment

To calculate the number of samples needed for each metric, I can use an [online calculator](http://www.evanmiller.org/ab-testing/sample-size.html). <br>

Alpha will be 5% which signifies a normal distribution with a 95% confidence interval <br>
Beta will be 20% which is typical <br>
Minimum Detectable Effect (dmin) will be 1% <br>
Baseline Conversion rate is dependent on the metric <br>


* **Gross Conversion** <br>
baseline = .20625 <br>
Sample size = 25,835 <br>
Total sample size = 25,835 x 2 = 51,670 <br>
(Because there is two groups experimental and control there is a multiple of 2) <br>
(sample size has the units of enrollments and has to be converted to pageviews) <br>
Enrollments per day/Unique cookies to click = clicks/Unique pageviews <br>
**Pageviews = 51,670/.08 = 645,875 **

* **Retention** <br>
baseline = .53 <br>
Sample size = 39,115 <br>
Total sample size = 39,115 x 2 = 78,230 <br>
(Because there is two groups experimental and control there is a multiple of 2) <br>
(sample size has the units of enrollments and has to be converted to pageviews) <br>
Enrollments per day/Unique cookies to click = Enrollments per day/Unique pageviews <br>
**Pageviews = 78,230/(660/40,000) = 4,741,213 **

* **Net Conversion** <br>
baseline = .109313 <br>
Sample size = 15,463 <br>
Total sample size = 15,463 x 2 = 30,926 <br>
(Because there is two groups experimental and control there is a multiple of 2) <br>
(sample size has the units of enrollments and has to be converted to pageviews) <br>
Enrollments per day/Unique cookies to click = clicks/Unique pageviews <br>
**Pageviews = 30,926/.08 = 386,575 **

## Methods to Increase Page Views
* Increase confidence leve (1-alpha) <br>
* Higher sensitivity (1-beta) <br>
* Decrease practitical significance level (minimum detectable effect, dmin) <br>

## Methods to Decrease Page Views
* Decrease confidence leve (1-alpha) <br>
* Lower sensitivity (1-beta) <br>
* Increase practitical significance level (minimum detectable effect, dmin) <br>

## Duration vs Exposure

Because the number of pageviews for retention is is 4.7 million which is significantly larger than the other metrics, it will be excluded in the test. <br>

The number of pageviews then needed for the experiment will be the next largest number which is 645,875 pageviews <br>

The experiment will only be conducted on a percentage of the traffic to the website such as 50%. The number of days needed to run this experiment is calculated by 645,875/(40,000 x .5) = 32.29 or **33 days** <br>
(40,000 is the baseline number of visitors per day)

# Step 4: Experiment Analysis

## Sanity Checks
To ensure that the experiment has valid results the confidence intervals for the control and experiments groups will be compared to the observed values in the experiment

The traffic is assumed to be divided between the control and experimentl group by 50:50 (p = .5)
With a 95% confidence interval the z-score will be 1.96

### Number of Cookies

In [13]:
controlc = control['Pageviews'].sum()
experimentc = experiment['Pageviews'].sum()
p = .5
z = 1.96
std = math.sqrt(p*(1-p)/(controlc +experimentc))
margin_of_error = z * std
lower_bound = p - margin_of_error
upper_bound = p + margin_of_error
observed = controlc/(controlc+experimentc)

print('The margin of error = ', margin_of_error)
print('The confidence interval is between ', lower_bound, ' and ',upper_bound)
print('The observed value is ', observed)

The margin of error =  0.0011796078509768765
The confidence interval is between  0.49882039214902313  and  0.5011796078509769
The observed value is  0.5006396668806133


Because the observed value is between the confidence interval the number of cookies passes the sanity check

### Number of Clicks

In [14]:
controlc = control['Clicks'].sum()
experimentc = experiment['Clicks'].sum()
p = .5
z = 1.96
std = math.sqrt(p*(1-p)/(controlc +experimentc))
margin_of_error = z * std
lower_bound = p - margin_of_error
upper_bound = p + margin_of_error
observed = controlc/(controlc+experimentc)

print('The margin of error = ', margin_of_error)
print('The confidence interval is between ', lower_bound, ' and ',upper_bound)
print('The observed value is ', observed)

The margin of error =  0.0041155042762105335
The confidence interval is between  0.49588449572378945  and  0.5041155042762105
The observed value is  0.5004673474066628


Because the observed value is between the confidence interval the number of clicks passes the sanity check

### Click-through-probability

Because this is a rate the standard deviation is defined as the sqrt(p_control(1-p_control)/n_experiment)

In [15]:
controlc = control['Clicks'].sum()/control['Pageviews'].sum()
experimentc = experiment['Clicks'].sum()/experiment['Pageviews'].sum()
n = experiment['Pageviews'].sum()
p = controlc
z = 1.96
std = math.sqrt(p*(1-p)/(n))
margin_of_error = z * std
lower_bound = p - margin_of_error
upper_bound = p + margin_of_error
observed = experimentc

print('The margin of error = ', margin_of_error)
print('The confidence interval is between ', lower_bound, ' and ',upper_bound)
print('The observed value is ', observed)

The margin of error =  0.0009166257431471257
The confidence interval is between  0.08120918783142969  and  0.08304243931772395
The observed value is  0.08218244066616376


Because the observed value is between the confidence interval the click-through-probability passes the sanity check

**All invariant metrics passed the sanity check so the experiment is valid to use and analyze**

## Results Analysis

### Gross Conversion

In [16]:
#only include clicks that have enrollments
n_control = control[control.Enrollments.notnull()]['Clicks'].sum()
n_experiment = experiment[experiment.Enrollments.notnull()]['Clicks'].sum()
x_control = control['Enrollments'].sum()
x_experiment = experiment['Enrollments'].sum()

p_pooled = (x_control + x_experiment)/(n_control + n_experiment)
se_pooled = math.sqrt(p_pooled*(1-p_pooled)*(1/n_control + 1/n_experiment))

d = (x_experiment/n_experiment) - (x_control/n_control)

lower_bound = d - (1.96 * se_pooled)
upper_bound = d + (1.96 * se_pooled)

print('P pooled = ', p_pooled)
print('Standard Deviation of p pooled = ', se_pooled)
print('Difference of the groups = ', d)
print('Confidence Interval is between ', lower_bound, 'and',upper_bound)

P pooled =  0.20860706740369866
Standard Deviation of p pooled =  0.004371675385225936
Difference of the groups =  -0.020554874580361565
Confidence Interval is between  -0.0291233583354044 and -0.01198639082531873


The confidence interval does not include zero, indicating that the result is **statistically significant**. <br>
The difference is not between -1% or 1%, which also means it is **practically significant** <br>
*1% was chosen as the practical significance earlier in the project when calculating sample size* <br>
*1.96 is the z-score for 95% confidence interval*

### Net Conversion

In [17]:
#only include clicks that have payments
n_control = control[control.Payments.notnull()]['Clicks'].sum()
n_experiment = experiment[experiment.Payments.notnull()]['Clicks'].sum()
x_control = control['Payments'].sum()
x_experiment = experiment['Payments'].sum()

p_pooled = (x_control+x_experiment)/(n_control+n_experiment)
se_pooled = math.sqrt(p_pooled*(1-p_pooled)*(1/n_control + 1/n_experiment))

d = (x_experiment/n_experiment) - (x_control/n_control)

lower_bound = d - (1.96 * se_pooled)
upper_bound = d + (1.96 * se_pooled)

print('P pooled = ', p_pooled)
print('Standard Deviation of p pooled = ', se_pooled)
print('Difference of the groups = ', d)
print('Confidence Interval is between ', lower_bound, 'and',upper_bound)

P pooled =  0.1151274853124186
Standard Deviation of p pooled =  0.0034341335129324238
Difference of the groups =  -0.0048737226745441675
Confidence Interval is between  -0.011604624359891718 and 0.001857179010803383


The confidence interval includes zero, indicating that the result is ** NOT statistically significant**. <br>
The difference is also between -1% or 1%, which  means it is **NOT practically significant** <br>
*1% was chosen as the practical significance earlier in the project when calculating sample size* <br>
*1.96 is the z-score for 95% confidence interval*

### Sign Test (Binomial)

Used to double check evaluation metrics by calculating a two-tailed p-value to determine significance.
* Differences calculated on a  *daily level* between the control and experiment groups
* Positive differences are successed
* Negative differences are failtures

In [31]:
#Daily Table
signs = pd.DataFrame()
signs['date'] = control[control.Payments.notnull()]['Date']

signs['gc control'] = control['Enrollments'] / control[control.Enrollments.notnull()]['Clicks']
signs['gc experiment'] = experiment['Enrollments'] / experiment[experiment.Enrollments.notnull()]['Clicks']
signs['nc control'] = control['Payments'] / control[control.Payments.notnull()]['Clicks']
signs['nc experiment'] = experiment['Payments'] / experiment[experiment.Payments.notnull()]['Clicks']

signs['gc sign'] = signs['gc experiment'] - signs['gc control']
signs['nc sign'] = signs['nc experiment'] - signs['nc control']

signs.head()

,date,gc control,gc experiment,nc control,nc experiment,gc sign,nc sign
0,"Sat, Oct 11",0.195051,0.153061,0.101892,0.049563,-0.041990,-0.052330
1,"Sun, Oct 12",0.188703,0.147771,0.089859,0.115924,-0.040933,0.026065
2,"Mon, Oct 13",0.183718,0.164027,0.104510,0.089367,-0.019691,-0.015144
3,"Tue, Oct 14",0.186603,0.166868,0.125598,0.111245,-0.019735,-0.014353
4,"Wed, Oct 15",0.194743,0.168269,0.076464,0.112981,-0.026474,0.036517


In [36]:
trials = len(experiment[experiment.Enrollments.notnull()])
gc_success = len(signs[signs['gc sign'] > 0])
nc_success = len(signs[signs['nc sign'] > 0])

print('Gross Conversion success rate is ', gc_success,'/',trials)
print('Net Conversion success rate is ', nc_success,'/',trials)

Gross Conversion success rate is  4 / 23
Net Conversion success rate is  10 / 23


In [45]:
gc_pvalue = scipy.stats.binom_test(gc_success,n = trials, p = 0.5)
nc_pvalue = scipy.stats.binom_test(nc_success,n = trials, p = 0.5)

print('Gross Conversion p-value is ', gc_pvalue)
print('Net Conversion p-value is ', nc_pvalue)

Gross Conversion p-value is  0.00259947776794
Net Conversion p-value is  0.677639484406


The Gross Conversion p-value is less than .05, indicating that it is significant whereas the Net Conversion p-value is greater than .05 indicating it is not significant

# Summary

An experiment was conducted to see the effect of when students know how much time is needed to complete a course before enrolling. 

Invariant metrics (cookies, clicks, click through probability) were chosen to ensure that the experiment was valid and Evaluation metrics (gross conversion, net conversion) were chosen in order to determine if the change was significant both statistically and practically.

All invariant metrics passed the sanity checks, indicating the experiment was safe to evaluate.

Gross conversions was significant in both the size and sign test, while Net conversions was not significant in those tests.

In order to launch the experiment, both Evaluation metrics had to be significant because the Bonferroni Correction was not used.
The Bonferroni Correction is a method to evaluate multiple metrics at the same time, however it produces very conservative numbers when metrics are correlated with each other.